## 3. 코드 예시 1
- 정규방정식을 활용한 릿지 회귀모델 구현
    - sklearn.linear_model.Ridge 클래스에서 'cholesky' solver 접근법에 해당.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
# 5.1 릿지 회귀모델 구현(정규방정식 풀이 방식, 촐레스키 접근법)

# boston 데이터셋 로드
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
boston = load_boston()
X, y = boston.data, boston.target

# X는 정규화 진행
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 1234)

# 정규 방정식 풀이
y_train = y_train.reshape(-1, 1) 
y_test = y_test.reshape(-1,1)

n_train, n_test = X_train.shape[0], X_test.shape[0] # 각각 339, 167개

X_train = np.append(np.ones((n_train, 1)), X_train, axis = 1)
X_test = np.append(np.ones((n_test, 1)), X_test, axis = 1)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i

In [3]:
# alpha 값 조정에 따른 효과
alphas = [0, 0.1, 1] # 알파값

for alpha in alphas :
    w = np.linalg.inv(X_train.T @ X_train + alpha *
                     np.identity(X_train.shape[1])) @ X_train.T @ y_train
    y_pred_train = X_train @ w
    print(f'alpha 값이 {alpha}일 때 :')
    print(f'학습 데이터셋 MAE : {np.abs(y_pred_train - y_train).mean() : .3f}')
    y_pred = X_test @ w
    print(f'테스트 데이터셋 MAE : {np.abs(y_pred - y_test).mean() : .3f}\n')

## 알파값 0일 때 OLS 추정값과 동일
## 알파 증가 -> 추정값 bias 증가 -> MAE 증가

alpha 값이 0일 때 :
학습 데이터셋 MAE :  3.328
테스트 데이터셋 MAE :  3.655

alpha 값이 0.1일 때 :
학습 데이터셋 MAE :  3.299
테스트 데이터셋 MAE :  3.580

alpha 값이 1일 때 :
학습 데이터셋 MAE :  3.250
테스트 데이터셋 MAE :  3.357



## 4. 코드 예시 2
- 릿지 클래스를 사용할 경우

In [4]:
# 5.2. 릿지 클래스 사용
from sklearn.linear_model import Ridge
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

boston = load_boston() # 사용할 데이터
X, y = boston.data, boston.target

# X는 정규화 진행
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 1234)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i

In [5]:
# alpha 값 조정에 따른 효과
alphas = [0, 0.1, 1] # 알파값

for alpha in alphas :
    reg = Ridge(alpha = alpha)
    reg = reg.fit(X_train, y_train)

    y_pred_train = reg.predict(X_train)
    print(f'alpha 값이 {alpha}일 때 :')
    print(f'학습 데이터셋 MAE : {np.abs(y_pred_train - y_train).mean() : .3f}')
    y_pred = reg.predict(X_test)
    print(f'테스트 데이터셋 MAE : {np.abs(y_pred - y_test).mean() : .3f}\n')

alpha 값이 0일 때 :
학습 데이터셋 MAE :  3.328
테스트 데이터셋 MAE :  3.655

alpha 값이 0.1일 때 :
학습 데이터셋 MAE :  3.312
테스트 데이터셋 MAE :  3.625

alpha 값이 1일 때 :
학습 데이터셋 MAE :  3.259
테스트 데이터셋 MAE :  3.486



#### 알파값 0에서는 차이가 없으나, 규제가 붙으면 이전 값들과 차이 발생. why?
- 둘 다 유클리디안 방식으로 정규화 함.
- 그러나 둘의 정규화 처리 방법에 차이가 있음.
    - 정규방정식 풀이 방식은 단순히 행렬 연산을 통해 해를 도출한 것.
        - 이렇다 할 최적화 알고리즘은 딱히 없음.
    - 반면, ridge 클래스는 내부적으로 최적화 알고리즘을 사용해 최적의 가중치를 찾은 것.